# TITLE

## Introduction

* Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
* Clearly state the question you will try to answer with your project
* Identify and describe the dataset that will be used to answer the question

## Preliminary exploratory data analysis

* Using only **training data**, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
* Using only **training data**, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

## Methods

* Explain how you will conduct either your data analysis and which variables/columns you will use. *Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?*
* Describe at least one way that you will visualize the results

## Expected outcomes and significance

* What do you expect to find?
* What impact could such findings have?
* What future questions could this lead to?

In [42]:
library(tidyverse)
library(tidymodels)
library(repr)
library(stringi)

**Step 1: reading and tidying the data**

In [46]:
player_stats <- read_csv("player_stats.csv")

names(player_stats) <- gsub(" ", "_", names(player_stats)) %>%
                        tolower()
player_stats$active[is.na(player_stats$active)] <- "No"

tidy_player_stats <- player_stats %>%
        
        separate(age, sep = " ", into = c("age", "birthdate"), convert = TRUE) %>%
                        
        separate(current_rank, sep = " ", into = c("current_rank", "current_rating"), convert = TRUE) %>%
        separate(best_rank, sep = " ", into = c("best_rank", "best_rank_date"), convert = TRUE) %>%
                        
        separate(favorite_surface, sep = " ", into = c("favorite_surface", "favorite_surface_percent")) %>%
        
        separate(current_elo_rank, sep = " ", into = c("current_elo_rank", "current_elo_rating"), convert = TRUE) %>% 
        separate(best_elo_rank, sep = " ", into = c("best_elo_rank", "best_elo_rank_date"), convert = TRUE) %>%

        separate(peak_elo_rating, sep = " ", into = c("peak_elo_rating", "peak_elo_rating_date"), convert = TRUE) %>%

        separate(goat_rank, sep = " ", into = c("goat_rank", "goat_rating"), convert = TRUE) %>%

        mutate(current_rating = parse_number(current_rating), 
                best_rank_date = str_replace(best_rank_date, "[(]", ""), best_rank_date = str_replace(best_rank_date, "[)]", ""), best_rank_date = as.Date(best_rank_date, format = '%d-%m-%Y'),
                height_cm = parse_number(height), weight_kg = parse_number(weight), prize_money = parse_number(prize_money), 
                current_elo_rating = parse_number(current_elo_rating),
                best_elo_rank_date = str_replace(best_elo_rank_date, "[(]", ""), best_elo_rank_date = str_replace(best_elo_rank_date, "[)]", ""), best_elo_rank_date = as.Date(best_elo_rank_date, format = '%d-%m-%Y'),
                peak_elo_rating_date = str_replace(peak_elo_rating_date, "[(]", ""), peak_elo_rating_date = str_replace(peak_elo_rating_date, "[)]", ""), peak_elo_rating_date = as.Date(peak_elo_rating_date, format = '%d-%m-%Y'),
                last_appearance = as.Date(last_appearance, format = '%d-%m-%Y'),
                goat_rating = parse_number(goat_rating),
                active = as.factor(active)) %>%
                    
        select(-x1, -wikipedia, -name, -facebook, -twitter, -nicknames, -birthdate, -coach, -web_site, -height, -weight, -favorite_surface_percent, -birthplace, -residence) %>%
                        
        arrange(current_rank) %>%
        filter(!is.na(current_rank))

glimpse(tidy_player_stats)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  `Turned Pro` = col_double(),
  Seasons = col_double(),
  Titles = col_double(),
  `Best Season` = col_double(),
  Retired = col_double(),
  Masters = col_double(),
  `Grand Slams` = col_double(),
  `Davis Cups` = col_double(),
  `Team Cups` = col_double(),
  Olympics = col_double(),
  `Weeks at No. 1` = col_double(),
  `Tour Finals` = col_double()
)

See spec(...) for full column specifications.

Warning message:
“Expected 2 pieces. Additional pieces discarded in 41 rows [3, 5, 6, 20, 21, 53, 58, 66, 89, 119, 129, 141, 142, 152, 155, 165, 188, 193, 205, 212, ...].”


Rows: 495
Columns: 34
$ age                  <int> 33, 32, 38, 23, 26, 21, 22, 23, 31, 33, 29, 32, …
$ country              <chr> "Spain", "Serbia", "Switzerland", "Russian Feder…
$ plays                <chr> "Left-handed", "Right-handed", "Right-handed", "…
$ current_rank         <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…
$ current_rating       <dbl> 10235, 9720, 6590, 5960, 5890, 5375, 3345, 2870,…
$ best_rank            <int> 1, 1, 1, 4, 4, 5, 3, 8, 9, 6, 7, 9, 13, 11, 3, 1…
$ best_rank_date       <date> 2008-08-18, 2011-07-04, 2004-02-02, 2019-09-09,…
$ backhand             <chr> "Two-handed", "Two-handed", "One-handed", "Two-h…
$ prize_money          <dbl> 119601561, 139144944, 129231891, 10507693, 22132…
$ favorite_surface     <chr> "Clay", "Fast", "Grass", "Hard", "Clay", "Clay",…
$ turned_pro           <dbl> 2001, 2003, 1998, 2014, 2011, 2016, 2013, 2015, …
$ seasons              <dbl> 19, 17, 22, 5, 10, 4, 8, 3, 11, 17, 10, 16, 5, 8…
$ active               <fct> Y

**Step 2: Splitting into training and test data**

In [41]:
set.seed(1)

stats_split <- initial_split(tidy_player_stats, prop = 0.75, strata = current_rank)
stats_training <- training(stats_split)
stats_testing <- testing(stats_split)